In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cp "./drive/MyDrive/CORD-1k-001.zip" '.'

In [ ]:
!unzip "CORD-1k-001.zip" 

Archive:  CORD-1k-001.zip
  inflating: CORD/dev/image/receipt_00074.png  
  inflating: CORD/dev/image/receipt_00099.png  
  inflating: CORD/dev/image/receipt_00073.png  
  inflating: CORD/dev/image/receipt_00063.png  
  inflating: CORD/dev/image/receipt_00005.png  
  inflating: CORD/dev/image/receipt_00049.png  
  inflating: CORD/dev/image/receipt_00075.png  
  inflating: CORD/dev/image/receipt_00070.png  
  inflating: CORD/dev/image/receipt_00067.png  
  inflating: CORD/dev/image/receipt_00077.png  
  inflating: CORD/dev/image/receipt_00010.png  
  inflating: CORD/dev/image/receipt_00071.png  
  inflating: CORD/dev/image/receipt_00062.png  
  inflating: CORD/dev/image/receipt_00048.png  
  inflating: CORD/dev/image/receipt_00013.png  
  inflating: CORD/dev/image/receipt_00061.png  
  inflating: CORD/dev/json/receipt_00066.json  
  inflating: CORD/dev/image/receipt_00076.png  
  inflating: CORD/train/json/receipt_00376.json  
  inflating: CORD/dev/image/receipt_00088.png  
  inflating:

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q datasets seqeval

     |████████████████████████████████| 362 kB 8.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 140 kB 69.0 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 1.1 MB 44.8 MB/s 
     |████████████████████████████████| 212 kB 53.2 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
     |████████████████████████████████| 127 kB 57.8 MB/s 
     |████████████████████████████████| 271 kB 84.5 MB/s 
     |████████████████████████████████| 144 kB 83.2 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 8.3 MB/s 


In [ ]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os 
import json
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
import torch
import datasets
from datasets import load_metric
import warnings
warnings.simplefilter('ignore')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
folders = ["train", "dev", "test"]

file_names_list = []

for folder in folders:
  #image
  file_images = os.listdir(f"./CORD/{folder}/image")
  #json
  file_jsons = os.listdir(f"./CORD/{folder}/json")
 
  names_image = []
  for p in file_images:
    name =  os.path.splitext(p)[0]
    names_image.append(name)

  names_json = []
  for p in file_jsons:
     name =  os.path.splitext(p)[0]
     names_json.append(name)

  set_names = set(names_image) & set(names_json)
  file_names = list(set_names)
  file_names.sort()

  file_names_list.append(file_names)

In [ ]:
#jsonfile一括読み込み関数
def loading_data(filepath, files_names):
  data_lists = []
  for file_name in files_names: 
     with open(f"{filepath}/{file_name}.json") as f:
       load_file = json.load(f)
       data_lists.append(load_file)
  return data_lists 

#imagedata読み込み関数
def loading_images(image_path, image_names):
  image_lists = []
  for name in image_names:
    image = Image.open(f"{image_path}/{name}.png").convert("RGB")
    image_lists.append(image)
  return image_lists

In [ ]:
json_lists = []
for i in range(len(folders)):
  json_lists.append(loading_data(f"./CORD/{folders[i]}/json", file_names_list[i]))

In [ ]:
image_lists = []
for i in range(len(folders)):
  image_lists.append(loading_images(f"./CORD/{folders[i]}/image", file_names_list[i]))

In [ ]:
#bounding boxを得る
def get_line_bbox(bboxs):
        x = [bbox[i] for i in bboxs.keys() if "x" in i]
        y = [bbox[i] for i in bboxs.keys() if "y" in i]

        x0, y0, x1, y1 = min(x), min(y), max(x), max(y)
        return [x0, y0, x1, y1]

In [ ]:
def normalize_bbox(bbox, size):
    x0 = int(1000*bbox[0] / size[0])
    y0 = int(1000*bbox[1] /size[1])
    x1 = int(1000*bbox[2] / size[0])
    y1 = int(1000*bbox[3] / size[1])
    if x0 > 1000: x0 = 1000
    if y0 > 1000: y0 = 1000
    if x1 > 1000: x1 = 1000
    if y1 > 1000: y1 = 1000
    if x0 < 0: x0 = 0
    if y0 < 0: y0 = 0
    if x1 < 0: x1 = 0
    if y1 < 0: y1 = 0
    return [x0, y0, x1, y1]
    # return [
    #     int(1000 * bbox[0] / size[0]),
    #     int(1000 * bbox[1] / size[1]),
    #     int(1000 * bbox[2] / size[0]),
    #     int(1000 * bbox[3] / size[1]),
    # ]

In [ ]:
datasets_list = {}

for k, folder in enumerate(folders):
  if len(image_lists[k]) != len(json_lists[k]):
    raise Exception('image画像とjsonデータの数が一致しません.')

  datasets = []

  for i in range(len(json_lists[k])):
    dataset ={"id": i, "tokens": "", "bboxes": "", "ner_tags": "", "image": image_lists[k][i]}
    words = []
    tokens = []
    category_lists = []
    bboxes = []
    #bboxの正規化用にimae.sizeを取得
    w, h = image_lists[k][i].size
    for group in json_lists[k][i]["valid_line"]:
      category = group["category"]
      words = group["words"]
      for word in words:
        bbox = word["quad"]
        text = word["text"]
        tokens.append(text)
        category_lists.append(category)
        bbox = get_line_bbox(bbox)
        #正規化
        norm_bbox = normalize_bbox(bbox, [w, h])
        bboxes.append(norm_bbox)
    dataset["tokens"] =tokens
    dataset["ner_tags"] = category_lists
    dataset["bboxes"] = bboxes
    datasets.append(dataset)
  datasets_list[folder] = datasets



In [ ]:
1000*(1298/ 1296)

1001.5432098765431

In [ ]:
datasets_list["train"][199]["bboxes"]

[[64, 252, 223, 291],
 [225, 255, 376, 295],
 [60, 291, 239, 334],
 [755, 307, 888, 344],
 [64, 335, 127, 375],
 [133, 336, 289, 375],
 [64, 378, 120, 415],
 [134, 378, 266, 418],
 [271, 377, 328, 418],
 [337, 381, 484, 419],
 [753, 385, 890, 423],
 [69, 422, 123, 462],
 [134, 421, 284, 461],
 [60, 461, 120, 504],
 [131, 461, 284, 500],
 [296, 459, 376, 502],
 [753, 463, 891, 501],
 [57, 504, 214, 545],
 [224, 502, 376, 543],
 [59, 547, 166, 586],
 [173, 545, 373, 586],
 [384, 541, 489, 584],
 [775, 543, 896, 583],
 [59, 590, 216, 629],
 [218, 585, 378, 628],
 [56, 631, 189, 671],
 [195, 626, 331, 670],
 [776, 623, 899, 664],
 [54, 675, 184, 717],
 [199, 668, 348, 716],
 [45, 716, 187, 763],
 [194, 713, 322, 757],
 [774, 705, 906, 750],
 [53, 762, 209, 807],
 [217, 756, 394, 801],
 [42, 805, 164, 851],
 [166, 802, 329, 848],
 [756, 788, 906, 836],
 [48, 847, 256, 895],
 [785, 831, 930, 876],
 [91, 893, 313, 937],
 [759, 873, 912, 920],
 [767, 959, 909, 1000]]

In [ ]:
label_list = []
for data in datasets_list["train"]:
  for label in data["ner_tags"]:
    label_list.append(label)

In [ ]:
def get_label_list(labels):
  unique_label = set(labels)
  unique_label = list(unique_label)
  unique_label.sort()
  id2label = {k: v for k,v in enumerate(unique_label)}
  label2id = {v: k for k,v in enumerate(unique_label)}
  return id2label, label2id

In [ ]:
id2label , label2id = get_label_list(label_list)

In [ ]:
label2id

{'menu.cnt': 0,
 'menu.discountprice': 1,
 'menu.etc': 2,
 'menu.nm': 3,
 'menu.num': 4,
 'menu.price': 5,
 'menu.sub_cnt': 6,
 'menu.sub_etc': 7,
 'menu.sub_nm': 8,
 'menu.sub_price': 9,
 'menu.sub_unitprice': 10,
 'menu.unitprice': 11,
 'menu.vatyn': 12,
 'sub_total.discount_price': 13,
 'sub_total.etc': 14,
 'sub_total.othersvc_price': 15,
 'sub_total.service_price': 16,
 'sub_total.subtotal_price': 17,
 'sub_total.tax_price': 18,
 'total.cashprice': 19,
 'total.changeprice': 20,
 'total.creditcardprice': 21,
 'total.emoneyprice': 22,
 'total.menuqty_cnt': 23,
 'total.menutype_cnt': 24,
 'total.total_etc': 25,
 'total.total_price': 26,
 'void_menu.nm': 27,
 'void_menu.price': 28}

In [ ]:
#traindataのラベルをid化
for  data in datasets_list["train"]:
  for i, label in enumerate(data["ner_tags"]):
    data["ner_tags"][i] = label2id[label]

In [ ]:
#dev_dataをidへ
for  data in datasets_list["dev"]:
  for i, label in enumerate(data["ner_tags"]):
    data["ner_tags"][i] = label2id[label]

#model

In [ ]:
from transformers import AutoProcessor

# we'll use the Auto API here - it will load LayoutLMv3Processor behind the scenes,
# based on the checkpoint we provide from the hub
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

In [ ]:
def prepare_examples(example):
  image = example["image"]
  words = example["tokens"]
  boxes = example["bboxes"]
  word_labels = example["ner_tags"]

  encoding = processor(image, words, boxes=boxes, word_labels=word_labels, 
                       truncation=True,  padding="max_length", return_tensors="pt")
  
  encoding = {k: v.squeeze()  for k, v in encoding.items()}

  return encoding


In [ ]:
train_dataset = list(map(prepare_examples, datasets_list["train"]))

In [ ]:
eval_dataset = list(map(prepare_examples, datasets_list["dev"]))

In [ ]:
import torch

example = eval_dataset[20]
for k, v in example.items():
  print(k, v.shape)

input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])
pixel_values torch.Size([3, 224, 224])


In [ ]:
processor.tokenizer.decode(eval_dataset[0]["input_ids"])

'<s> 1 REAL GANACHE 16,500 1 EGG TART 13,000 1 PIZZA TOAST 16,000 TOTAL 45,500 CASH 50,000 CHANGE 4,500</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

## Define metrics
#

#Define model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Tue Jun 28 06:52:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#create dataLoader
train_dataLoader = DataLoader(train_dataset, batch_size=2, shuffle=True)
eval_dataLoader = DataLoader(eval_dataset, batch_size = 3)

In [ ]:
#compute f1score
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrix(logits, labels):
  predictions = torch.argmax(logits, axis=2)
  true_predictions = [
            [id2label[p.item()] for (p, l) in zip (prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
               [id2label[l.item()] for (p,l) in zip(prediction, label) if l != -100]
               for prediction, label in zip(predictions, labels)
  ]

  results = metric.compute(predictions=true_predictions, references=true_labels)


  return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }


In [ ]:
#optimizer 
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
#create rate scheduler
from transformers import get_scheduler

num_epochs = 8
num_training_steps = num_epochs * len(train_dataLoader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


LayoutLMv3ForTokenClassification(
  (layoutlmv3): LayoutLMv3Model(
    (embeddings): LayoutLMv3TextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
    )
    (patch_embed): LayoutLMv3PatchEmbeddings(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (encoder): LayoutLMv3Encoder

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))




model.train()
for epoch in range(num_epochs):
  results = {"precision": [], "recall":[], "f1": [], "accuracy":[]}
  for i, batch in enumerate(train_dataLoader):
    batch = {k: v.to(device) for k, v in batch.items()}
    labels = batch["labels"]
    output = model(**batch)
    result = compute_metrix(output.logits, labels)
    for k, v in result.items():
      results[k].append(v)
    loss = output.loss
    loss.backward()
    if (i + 1)%32 == 0:
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
    progress_bar.update(1)
  print("epoch数: ", epoch)
  for k, v in result.items():
    print(k, sum(results[k]) / len(results[k]))
  print("-----------------------------------------")

  0%|          | 0/2936 [00:00<?, ?it/s]

epoch数:  0
precision 0.29337559416449277
recall 0.30476989475418875
f1 0.28989721141519026
accuracy 0.37280061916906204
-----------------------------------------
epoch数:  1
precision 0.4274286066211105
recall 0.4923380810736822
f1 0.4552770854942862
accuracy 0.5676285754556258
-----------------------------------------
epoch数:  2
precision 0.5850577491675145
recall 0.6328860992009621
f1 0.6061991463768
accuracy 0.7192581750664556
-----------------------------------------
epoch数:  3
precision 0.7620857965249671
recall 0.7761812123052942
f1 0.767853586729212
accuracy 0.8152192891167589
-----------------------------------------
epoch数:  4
precision 0.8512584837789943
recall 0.8478957038611
f1 0.8489756931564353
accuracy 0.8762356834952943
-----------------------------------------
epoch数:  5
precision 0.8890757269390488
recall 0.8854030307959352
f1 0.8866616320935071
accuracy 0.9080470784820512
-----------------------------------------
epoch数:  6
precision 0.913516398298418
recall 0.9146549

In [ ]:
progress_bar = tqdm(range(len(eval_dataLoader)))
results = {"precision": [], "recall":[], "f1": [], "accuracy":[]}

for batch in eval_dataLoader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    output = model(**batch)
    labels = batch["labels"]
    result = compute_metrix(output.logits, labels)
    for k, v in result.items():
      results[k].append(v)
    progress_bar.update(1)

for k, v in result.items():
  print(k, sum(results[k]) / len(results[k]))


  0%|          | 0/31 [00:00<?, ?it/s]

precision 0.9218634339993668
recall 0.928595070220707
f1 0.9249181863533662
accuracy 0.935958668031526
